In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [5]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("kiwibm25_faiss_37")

LangSmith 추적을 시작합니다.
[프로젝트명]
kiwibm25_faiss_37


## 본 코드

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt')
docs = loader.load()

In [7]:
docs

[Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt'}, page_content='메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다\n\n메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다\n\n메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다\n\n메뉴이름  토마토 치즈 비프 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 신선한 토마토와 고소한 치즈버거의 조화 이다 식재료는 난류우유대두밀토마토쇠고기가 들어간다 중량은 200g이다 열량은 403kcal이다 단품가격은 4600원 세트가격은 6900원이다\n\n메뉴이름  더블 쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 좋아하는건 더 많이 즐기시라고 두 배 이상 커진  파운드 비프 패티가 두 장 그램육즙이 풍부한 고기 맛을 그대로 살린  순 쇠고기 패티 두 장과 치즈 두 장이 입안 가득 완벽하게 조화되는 놀라운 맛 이다 식재료는 우유

In [8]:
from langchain.text_splitter import CharacterTextSplitter

# CharacterTextSplitter를 사용하여 문서 분할
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 173, which is longer than the specified 100
Created a chunk of size 163, which is longer than the specified 100
Created a chunk of size 212, which is longer than the specified 100
Created a chunk of size 143, which is longer than the specified 100
Created a chunk of size 236, which is longer than the specified 100
Created a chunk of size 242, which is longer than the specified 100
Created a chunk of size 208, which is longer than the specified 100
Created a chunk of size 198, which is longer than the specified 100
Created a chunk of size 171, which is longer than the specified 100
Created a chunk of size 201, which is longer than the specified 100
Created a chunk of size 187, which is longer than the specified 100
Created a chunk of size 173, which is longer than the specified 100
Created a chunk of size 182, which is longer than the specified 100
Created a chunk of size 153, which is longer than the specified 100
Created a chunk of size 175, which is longer tha

In [9]:
print("분할된 문서 수:", len(split_docs))

분할된 문서 수: 27


In [10]:
from kiwipiepy import Kiwi
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [11]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")  # OpenAI 임베딩을 사용합니다.
store = LocalFileStore("./.cache2")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

faiss_vectorstore = FAISS.from_documents(
    split_docs,
    cached_embedder,
)
faiss = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

In [12]:
kiwi_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=kiwi_tokenize)
kiwi_bm25.k = 4

kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

In [13]:
results = kiwibm25_faiss_37.invoke("닭고기가 들어간 버거를 추천해줘")

for i in results:

    print(i.page_content)
    print("-"*100)

메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
----------------------------------------------------------------------------------------------------
메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
----------------------------------------------------------------------------------------------------
메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
----------------------------------------------------------------------------------------------------
메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 단품가격은 7600원

# LLM연결

In [19]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)

qa = RetrievalQA.from_chain_type(
    llm = openai,
    chain_type = "stuff",
    retriever = kiwibm25_faiss_37,
    return_source_documents = True
)

def test(query):
    result = qa(query)
    print(f"LLM 답변: {result['result']}")
    print("="*50)

    for i in result['source_documents']:
        print(f"검색 결과 문서: {i.page_content}")
        print("-"*50)


## 테스트

In [20]:
# 검색 결과 문서를 가져옵니다.
query = "닭고기가 들어간 버거를 추천해줘"
test(query)

LLM 답변: 맥치킨 모짜렐라 버거와 맥크리스피 스리라차 마요 버거는 둘 다 닭고기가 들어간 맛있는 버거입니다.
검색 결과 문서: 메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 단품가격은 7600원 세트가격은 9200원이다
--------------------------------

In [21]:
query = "닭고기가 들어가지 않은 버거를 추천해줘"
test(query)

LLM 답변: 제가 알고 있는 정보에 따르면, 닭고기가 들어가지 않은 버거는 "더블 불고기 버거"입니다. 이 버거는 불고기 소스의 패티가 두 장이 들어가고, 고소한 치즈와 마요네즈, 신선한 양상추가 함께 제공됩니다.
검색 결과 문서: 메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 

In [22]:
query = "신메뉴는 뭐가있어?"
test(query)

LLM 답변: 저는 그 정보를 알 수 없습니다.
검색 결과 문서: 메뉴이름  탄산 미디움 카테고리는 음료 이며 신메뉴가 아니다 이 메뉴는 갈증해소 뿐만이 아니라 기분까지 상쾌하게 코카콜라 이다 단품가격은 2600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  슈슈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 탱글한 통새우살 가득 슈슈 버거 이다 식재료는 난류우유대두밀토마토새우쇠고기굴가 들어간다 중량은 224g이다 열량은 409kcal이다 단품가격은 5500원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  슈비 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 탱글한 통새우살에 비프 패티를 더해 푸짐하게 이다 식재료는 난류우유대두밀토마토새우쇠고기굴가 들어간다 중량은 269g이다 열량은 540kcal이다 단품가격은 6600원 세트가격은 9100원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기

In [23]:
query = "매콤한 메뉴는 뭐가있어?"
test(query)

LLM 답변: 매콤한 메뉴로는 맥크리스피 스리라차 마요 버거, 베토디 스리라차 마요 버거, 그리고 베이컨 토마토 디럭스 버거가 있습니다.
검색 결과 문서: 메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 단품가격은 7600원 세트가격은 9200원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 클래식 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 치킨 버거 본연의 맛에 충실한 클래식 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 210g이다 열량은 603kcal이다 단품가격은 6700원 세트가격은 8700원이다
---------------------

In [24]:
query = "매콤하지 않은 메뉴는 뭐가있어?"
test(query)

LLM 답변: 매콤하지 않은 메뉴로는 "불고기 버거"가 있습니다.
검색 결과 문서: 메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 클래식 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 치킨 버거 본연의 맛에 충실한 클래식 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 210g이다 열량은 603kcal이다 단품가격은 6700원 세트가격은 8700원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 단품가격은 7600원 세트가격은 9200원이다
-------------------------------------------------

In [25]:
query = "가장 열량이 낮은 메뉴는 뭐가있어?"
test(query)

LLM 답변: 열량이 가장 낮은 메뉴는 "불고기 버거"입니다. 해당 메뉴의 열량은 409kcal이며, 다른 메뉴들보다 낮은 열량을 가지고 있습니다.
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
--------------------------------------------------
검색 결과

In [26]:
query = "가장 양이 적은 버거는 뭐야?"
test(query)

LLM 답변: 양이 적은 버거는 "치즈버거"입니다.
검색 결과 문서: 메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다 단품가격은 7200원 세트가격은 9100원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  치즈버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 고소하고 부드러운 치즈와  순 쇠고기 패티 맥도날드만의 심플한 클래식 치즈버거  이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 115g이다 열량은 318kcal이다 단품가격은 3600원 세트가격은 6300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  빅맥 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는  순 쇠고기 패티 두 장에 빅맥만의 특별한 소스 입안에서 살살 녹는 치즈와 신선한 양상추 양파 그리고  피클까지 년 넘게 전 세계인의 입맛을 사로 잡은 버거의 대명사  이다 식재료는 난류우유대두밀쇠고기가 들어간다 중량은 223g이다 열량은 582kcal이다 단품가격은 6300원 세트가격은 8500원이다
-------------------------------

In [27]:
query = "해산물이 들어간 버거는 뭐가있어?"
test(query)

LLM 답변: 해산물이 들어간 버거는 "슈비 버거"와 "슈슈 버거"가 있습니다.
검색 결과 문서: 메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  슈비 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 탱글한 통새우살에 비프 패티를 더해 푸짐하게 이다 식재료는 난류우유대두밀토마토새우쇠고기굴가 들어간다 중량은 269g이다 열량은 540kcal이다 단품가격은 6600원 세트가격은 9100원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  슈슈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 탱글한 통새우살 가득 슈슈 버거 이다 식재료는 난류우유대두밀토마토새우쇠고기굴가 들어간다 중량은 224g이다 열량은 409kcal이다 단품가격은 5500원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드

In [28]:
query = "제일 싼 메뉴는 뭐야?"
test(query)

LLM 답변: 가장 저렴한 메뉴는 "후렌치후라이 스몰"이며, 단품가격은 2300원입니다.
검색 결과 문서: 메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  슈슈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 탱글한 통새우살 가득 슈슈 버거 이다 식재료는 난류우유대두밀토마토새우쇠고기굴가 들어간다 중량은 224g이다 열량은 409kcal이다 단품가격은 5500원 세트가격은 7300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다 단품가격은 7200원 세트가격은 9100원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  불고기 버거 카테고리

In [29]:
query = "제일 싼 버거는 뭐야?"
test(query)

LLM 답변: 가격을 기준으로 제일 싼 버거는 "맥치킨 버거"입니다. 해당 메뉴의 단품가격은 4300원이며, 세트가격은 6700원입니다.
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다 단품가격은 7200원 세트가격은 9100원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
--------------

In [30]:
query = "세트가격이 제일 싼 버거는 뭐야?"
test(query)

LLM 답변: 세트가격이 제일 싼 버거는 "치즈버거"야.
검색 결과 문서: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  치즈버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 고소하고 부드러운 치즈와  순 쇠고기 패티 맥도날드만의 심플한 클래식 치즈버거  이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 115g이다 열량은 318kcal이다 단품가격은 3600원 세트가격은 6300원이다
--------------------------------------------------
검색 결과 문서: 메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다 단품가격은 7200원 세트가격은 9100원이다
--------------------------------------------------
검색 